<center><h1>Creating PostgreSQL Table from a Pandas DataFrame</h1></center>

This example will show you how to create a PostgreSQL table consisting of customer complaints filed with the NHTSA (National Highway Traffic Safety Administration).  The metadata on NHTSA's text file (information about the data), can be viewed [here](http://www-odi.nhtsa.dot.gov/downloads/folders/Complaints/CMPL.txt).<br><br>The usual "best practice" and quickest to import a text file is to use the tools native to the database.  If using PostgreSQL, you would use its COPY command to import the text file as a table.  However, I would get the dreaded "extra data after last expected column" error when trying to import the complaints file "FLAT_CMPL.txt" from NHTSA's site.  The cause is due to malformed or incorrectly delimited row(s) in the data.  Unfortunately, "dirty" data is very common.<br><br>The usual workaround that others would suggest is to remove or not include the offending row(s) of data.  However, with a large text file, this is very cumbersome to do and may take a long time as you would have to identify the offending rows of data. I have learned that there is a tool called [pgloader](https://pgloader.io) that essentially loads rows of data in batches and allows you to manage errors during the COPY execution per its [documentation's](https://manpages.debian.org/stretch/pgloader/pgloader.1) "BATCHES AND RETRY BEHAVIOR":

### pgloader: BATCHES AND RETRY BEHAVIOUR
>To load data to PostgreSQL, pgloader uses the COPY streaming protocol. While this is the faster way to load data, COPY has an important drawback: as soon as PostgreSQL emits an error with any bit of data sent to it, whatever the problem is, the whole data set is rejected by PostgreSQL.
To work around that, pgloader cuts the data into batches of 25000 rows each, so that when a problem occurs it´s only impacting that many rows of data. Each batch is kept in memory while the COPY streaming happens, in order to be able to handle errors should some happen.

>When PostgreSQL rejects the whole batch, pgloader logs the error message then isolates the bad row(s) from the accepted ones by retrying the batched rows in smaller batches.

Sadly however, pgloader is not well supported on [Windows](https://github.com/dimitri/pgloader/issues/652).

### So...what other work-arounds, methods, or tools can be used to address malformed rows of data during a COPY command or file import for the creation of a PostgreSQL table?

I know that I could use Python's ContextManager ```with``` syntax to process the text file's contents, one line at a time, to identify malformed rows or lines of data:

```
with open(...) as f:
    for line in f:
        # Check line for malformed data, if found to be bad, drop it, etc
```

But, there could potentially be 2 or more different ways that a row or line of data is "bad" and therefore, I could potentially need to have several IF/ELSE branchings for every way that the row is bad or malformed.

## One Solution:

The pandas library's ```read_csv()``` function has a SUPER useful parameter called ```chunks=``` which allows you to process a fixed amount of rows at a time and apply complex processing if needed.  It also has an ```error_bad_lines=False``` parameter option which basically means it will skip or not include the offending rows of data.  With this method, larger-than-RAM text file can be processed and imported as a PostgreSQL table even if there are malformed rows of data.

The following demonstrates the method above.

## Table Creation Process Overview

1. Download the zip file containing the complaints text file from NHTSA's web site and then extract it
2. Define column names or header row for the text file since NHTSA did not include it in the text file
3. Define functions to create 3 new columns to represent the component tiers
4. Read in and then process the text file 20K rows at a time, creating 3 additional columns

### 1) Download the zip file in memory, then extract

Download the zip file and hold it in memory instead of physically writing/creating a file on disk.  Then unzip the contents of the zip file onto an actual folder location.

In [ ]:
%%time
from io import BytesIO
from zipfile import ZipFile
from urllib import request

url = request.urlopen('http://www-odi.nhtsa.dot.gov/downloads/folders/Complaints/FLAT_CMPL.zip')
zipfile_in_memory = ZipFile(BytesIO(url.read()))
zipfile_in_memory.extractall(r'D:\temp')
zipfile_in_memory.close()
print("Download and extraction completed")

### 2) Define column names for the table to be created since NHTSA inconveniently left it out of their csv file

Below I've created a list of column names that will be used for the ```complaints``` table in the PostgreSQL database:

In [1]:
columns = [
    'cmplid',
    'odino',
    'mfr_name',
    'maketxt',
    'modeltxt',
    'yeartxt',
    'crash',
    'faildate',
    'fire',
    'injured',
    'deaths',
    'compdesc',
    'city',
    'state',
    'vin',
    'datea',
    'ldate',
    'miles',
    'occurrences',
    'cdescr',
    'cmpl_type',
    'police_rpt_yn',
    'purch_dt',
    'orig_owner_yn',
    'anti_brakes_yn',
    'cruise_cont_yn',
    'num_cyls',
    'drive_train',
    'fuel_sys',
    'fuel_type',
    'trans_type',
    'veh_speed',
    'dot',
    'tire_size',
    'loc_of_tire',
    'tire_fail_type',
    'orig_equip_yn',
    'manuf_dt',
    'seat_type',
    'restraint_type',
    'dealer_name',
    'dealer_tel',
    'dealer_city',
    'dealer_state',
    'dealer_zip',
    'prod_type',
    'repaired_yn',
    'medical_attn',
    'vehicles_towed_yn'
]

**The vehicle component column ```compdesc``` contains a high level description of the component on the vehicle that failed.  However, within the description, there are hierarchies of the components that are denoted by the colon (:) symbol which could be useful for filtering purposes.<br><br>Example: "AIR BAGS:FRONTAL:SENSOR/CONTROL MODULE"<br><br>Although, this column is useful, our analysts would like to be able to filter by the components' sub-hierarchies or tiers also.**

**To facilitate this, we thought it would be a good idea to create additional columns to represent the component hierarchies or tiers.  It was decided to create 3 new columns that represent the component tiers at 3 levels.**

### 3) Define functions to create 3 new columns to represent the component tiers

Functions below are simply splitting the text based on any colons (:) found in the column of data. If no colon is found, then just return or keep the same value.

In [2]:
def tier1(column):
    if ':' not in column:
        return column
    else:
        # else return first item in a list of strings
        return column.split(':')[0].strip()

In [3]:
def tier2(column):
    if ':' not in column:
        return ''
    else:
        if len(column.split(':')) == 1:
            return ''
        else:
            # else return 2nd item in list of strings
            return column.split(':')[1].strip()

In [4]:
def tier3(column):
    if ':' not in column:
        return ''
    else:
        if len(column.split(':')) == 1:
            return ''
        elif len(column.split(':')) == 2:
            return ''
        else:
            # else return 3rd itme in list of strings
            return column.split(':')[2].strip()

### 3) Read in the text file, in chunks of 20K rows at a time, create 3 new columns, and then populate the ```complaints``` table with them

Using Pandas read_csv()'s ```chunksize``` parameter and ```error_bad_lines=False``` parameter option, we can process and import the text file even if it has malformed or incorrectly delimited rows of data.  This method also allows us to include 3 new columns, which would otherwise be more difficult to do using SQL.

In [5]:
from sqlalchemy import create_engine
import pandas as pd
import datetime as dt

disk_engine = create_engine("postgres://postgres:@VCLO49529.am.mds.honda.com:5432/NHTSA")

print("Dropping / Deleting complaints table if it exists...")
with disk_engine.connect() as conn:
    conn.execute("DROP TABLE IF EXISTS complaints")

chunksize = 20000
j = 0

# Only needed if you want to include your dataframe index, but starting at 1 instead of 0
# index_start = 1

begin = dt.datetime.now()

print('Creating the complaints table.  Please wait...')

# use the columns list to define the column names of the complaints table
for df in pd.read_csv(r'D:\temp\FLAT_CMPL.txt', chunksize=chunksize, names=columns, dtype="category",
                      delimiter='\t', iterator=True, encoding='ISO-8859-1', error_bad_lines=False):
    
    df = df.assign(comp_tier1 = df['compdesc'].map(tier1))
    df = df.assign(comp_tier2 = df['compdesc'].map(tier2))
    df = df.assign(comp_tier3 = df['compdesc'].map(tier3))
    
    # Only needed if you want to include your dataframe index, but starting at 1 instead of 0
    # df.index += index_start
    
    j+=1
    # To print on same line, use '\r' and end='' option with the print function
    print('\r'+'{} seconds: completed {} rows'.format((dt.datetime.now() - begin).seconds, j*chunksize),end='')

    df.to_sql('complaints', disk_engine, if_exists='append', index=False)
    
    # Only needed if you want to include your dataframe index, but starting at 1 instead of 0
    # index_start = df.index[-1] + 1

Creating the complaints table.  Please wait...
787 seconds: completed 1600000 rows

In [6]:
print("Total elapsed time (hr:min:sec):", dt.datetime.now() - begin)

Total elapsed time (hr:min:sec): 0:13:14.115472


### To improve query performance, let's create indices based on most frequently used columns used for filtering

Based on my past querying history, I've been filtering a lot based on combinations of year, make, model, and component description.

In [7]:
%%time
with disk_engine.connect() as conn:
    conn.execute('CREATE INDEX yearidx ON complaints (yeartxt)')
    conn.execute('CREATE INDEX makeidx ON complaints (maketxt)')
    conn.execute('CREATE INDEX modelidx ON complaints (modeltxt)')
    conn.execute('CREATE INDEX dateaidx ON complaints (datea)')
    conn.execute('CREATE INDEX compidx ON complaints (compdesc)')
    conn.execute('CREATE INDEX comp1idx ON complaints (comp_tier1)')
    conn.execute('CREATE INDEX comp2idx ON complaints (comp_tier2)')
    conn.execute('CREATE INDEX comp3idx ON complaints (comp_tier3)')

Wall time: 1min 27s


### Now with our complaints table created from the text file, let's see if our component tier columns were successfully created:

In [8]:
sql = """
SELECT
    compdesc,
    comp_tier1,
    comp_tier2,
    comp_tier3
    
FROM
    complaints
    
WHERE
    maketxt in('HONDA', 'ACURA')

LIMIT 50
"""

In [9]:
df = pd.read_sql_query(sql, disk_engine)

In [10]:
df

,compdesc,comp_tier1,comp_tier2,comp_tier3
0,AIR BAGS:FRONTAL,AIR BAGS,FRONTAL,
1,STEERING:WHEEL AND HANDLE BAR,STEERING,WHEEL AND HANDLE BAR,
2,AIR BAGS:FRONTAL,AIR BAGS,FRONTAL,
3,VISIBILITY:WINDSHIELD,VISIBILITY,WINDSHIELD,
4,STRUCTURE:BODY:DOOR,STRUCTURE,BODY,DOOR
5,WHEELS,WHEELS,,
6,VISIBILITY:DEFROSTER/DEFOGGER SYSTEM,VISIBILITY,DEFROSTER/DEFOGGER SYSTEM,
7,TIRES:TREAD/BELT,TIRES,TREAD/BELT,
8,"FUEL SYSTEM, GASOLINE:STORAGE:TANK ASSEMBLY","FUEL SYSTEM, GASOLINE",STORAGE,TANK ASSEMBLY
9,SUSPENSION:FRONT,SUSPENSION,FRONT,
